# <p style="text-align: center;"> Final Project - Introduction to Data Science</p>
# <p style="text-align: center;"> <b> Data Modeling </b></p>
---

## Member Information
| Name              | ID       |
|-------------------|----------|
|Tran Dinh Quang    | 21127406 |
|Nguyen Hong Hanh   | 21127503 |
|Do Quoc Tri  | 21127556 |
| Nguyen Khanh Nhan | 21127657         |


## Table of contents



---

## 0. Import libraries

## 1. Problem Statement

### 1.1 State the problem

### 1.2 Purpose of solving the problem

## 2. Data preparation

### 2.1 Pre-processing

### 2.2 Splitting data

## 3. Data Modeling

### 3.1 Model selection

### 3.2 Fine-tuning model

### 3.3 Model training

### 3.4 Basic ensemble method

### 3.5 Model evaluation